In [1]:
import pandas as pd
import orjson as json

In [13]:
for line in open('../data/train_matches.jsonl'):
    game = json.loads(line)
    print(game['game_time'])
    for player in game['players']:
        print(player['level'], player['deaths'], player['gold'], player['gold_reasons'], player['gold_t'])
    break

155
2 0 543 {'0': 600, '13': 308} [1, 91, 356]
2 0 399 {'0': 600, '13': 164} [1, 91, 262]
1 1 304 {'0': 669, '1': -67} [1, 91, 182]
2 0 389 {'0': 600, '13': 154} [1, 169, 260]
2 0 402 {'0': 600, '13': 167} [1, 91, 281]
3 0 982 {'0': 760, '12': 34, '13': 553} [1, 331, 738]
3 0 788 {'0': 760, '13': 393} [1, 251, 561]
2 0 531 {'0': 896} [1, 251, 342]
2 0 796 {'0': 829, '12': 332} [1, 251, 342]
3 0 851 {'0': 760, '13': 456} [1, 425, 680]


In [27]:
for line in open('../data/train_matches.jsonl'):
    game = json.loads(line)
    print(game['game_time'])
    for player in game['players']:
        print(player['level'], player['deaths'], player['xp'], player['xp_reasons'])
    break

155
2 0 533 {'2': 533}
2 0 478 {'2': 478}
1 1 130 {'2': 130}
2 0 506 {'2': 506}
2 0 344 {'2': 344}
3 0 780 {'1': 30, '2': 750}
3 0 706 {'2': 706}
2 0 307 {'2': 307}
2 0 421 {'1': 30, '2': 391}
3 0 870 {'2': 870}


In [5]:
player['gold'], player['gold_reasons']

(543, {'0': 600, '13': 308})

In [14]:
unique_gold_reasons = dict()
for file in ['../data/train_matches.jsonl', '../data/test_matches.jsonl']:
    for line in open(file):
        game = json.loads(line)
        for p in game['players']:
            for gr, v in p['gold_reasons'].items():
                if gr in unique_gold_reasons:
                    unique_gold_reasons[gr] += v
                else:
                    unique_gold_reasons[gr] = v

In [19]:
sorted(list(map(int, unique_gold_reasons.keys())))

[0, 1, 2, 5, 6, 11, 12, 13, 14, 15]

In [16]:
[(k, len(str(v))) for k, v, in unique_gold_reasons.items()]

[('0', 9),
 ('13', 10),
 ('1', 10),
 ('12', 9),
 ('6', 8),
 ('11', 9),
 ('5', 7),
 ('2', 9),
 ('15', 7),
 ('14', 8)]

In [30]:
def gold_features(player):
    if 'init' not in dir(gold_features):
        gold_features.gold_reasons = [0, 1, 2, 5, 6, 11, 12, 13, 14, 15]
        gold_features.init = {'net_gold': 0}
        gold_features.init.update({f'gold_reason_{i}': 0 for i in gold_features.gold_reasons})
        gold_features.init.update({f'gold_reason_{i}_frac': 0 for i in gold_features.gold_reasons})
    c = gold_features.init.copy()
    for reason, value in player['gold_reasons'].items():
        if reason == '0':
            value -= 600  # base value
        c['net_gold'] += value
        c['gold_reason_' + reason] = value
    for reason, value in player['gold_reasons'].items():
        c['gold_reason_' + reason + '_frac'] = c['gold_reason_' + reason] / c['net_gold']
    return c

In [31]:
gold_features(player)

{'net_gold': 616,
 'gold_reason_0': 160,
 'gold_reason_1': 0,
 'gold_reason_2': 0,
 'gold_reason_5': 0,
 'gold_reason_6': 0,
 'gold_reason_11': 0,
 'gold_reason_12': 0,
 'gold_reason_13': 456,
 'gold_reason_14': 0,
 'gold_reason_15': 0,
 'gold_reason_0_frac': 0.2597402597402597,
 'gold_reason_1_frac': 0,
 'gold_reason_2_frac': 0,
 'gold_reason_5_frac': 0,
 'gold_reason_6_frac': 0,
 'gold_reason_11_frac': 0,
 'gold_reason_12_frac': 0,
 'gold_reason_13_frac': 0.7402597402597403,
 'gold_reason_14_frac': 0,
 'gold_reason_15_frac': 0}

In [24]:
unique_xp_reasons = dict()
for file in ['../data/train_matches.jsonl', '../data/test_matches.jsonl']:
    for line in open(file):
        game = json.loads(line)
        for p in game['players']:
            for gr, v in p['xp_reasons'].items():
                if gr in unique_xp_reasons:
                    unique_xp_reasons[gr] += v
                else:
                    unique_xp_reasons[gr] = v

In [26]:
[(k, len(str(v))) for k, v in unique_xp_reasons.items()]

[('2', 10), ('1', 10), ('0', 9), ('3', 8)]

In [34]:
def xp_features(player):
    if 'init' not in dir(xp_features):
        xp_features.xp_reasons = [0, 1, 2, 3]
        xp_features.init ={f'xp_reason_{i}': 0 for i in xp_features.xp_reasons}
        xp_features.init.update({f'xp_reason_{i}_frac': 0 for i in xp_features.xp_reasons})
    c = xp_features.init.copy()
    for reason, value in player['xp_reasons'].items():
        c['xp_reason_' + reason] = value
        c['xp_reason_' + reason + '_frac'] = value / player['xp']
    return c

In [35]:
xp_features(player)

{'xp_reason_0': 0,
 'xp_reason_1': 0,
 'xp_reason_2': 870,
 'xp_reason_3': 0,
 'xp_reason_0_frac': 0,
 'xp_reason_1_frac': 0,
 'xp_reason_2_frac': 1.0,
 'xp_reason_3_frac': 0}